<a href="https://colab.research.google.com/github/chonchuks/Network_Anamoly_Detection/blob/main/test_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install fastapi
! pip install pyngrok
! pip install 'uvicorn[standard]'
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the path to the zip file
zip_file_path = '/content/drive/Othercomputers/My PC/models/Fastapi_Anomaly_detection.zip'

# Define the directory to unzip to
unzip_dir = '/content'

# Create the directory if it doesn't exist
os.makedirs(unzip_dir, exist_ok=True)

# Unzip the file to the specified directory
!unzip -q "{zip_file_path}" -d "{unzip_dir}"

unzip:  cannot find or open /content/drive/Othercomputers/My PC/models/Fastapi_Anomaly_detection.zip, /content/drive/Othercomputers/My PC/models/Fastapi_Anomaly_detection.zip.zip or /content/drive/Othercomputers/My PC/models/Fastapi_Anomaly_detection.zip.ZIP.


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import pandas as  pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)


# Load the trained LSTM model
model = tf.keras.models.load_model('best_lstm_model.h5')
le = LabelEncoder()
# Load StandardScaler and its parameters
scaler = StandardScaler()
scaler.mean_ = np.load('scaler_mean.npy')
scaler.scale_ = np.load('scaler_scale.npy')

# Define input data model
class ModelInput(BaseModel):
    duration: float
    protocoltype: str
    service: str
    flag: str
    srcbytes: int
    dstbytes: int
    wrongfragment: int
    hot: int
    loggedin: int
    numcompromised: int
    rootshell: int
    suattempted: int
    numroot: int
    numfilecreations: int
    numshells: int
    numaccessfiles: int
    ishostlogin: int
    isguestlogin: int
    count: int
    srvcount: int
    serrorrate: float
    srvserrorrate: float
    rerrorrate: float
    srvrerrorrate: float
    samesrvrate: float
    diffsrvrate: float
    srvdiffhostrate: float
    dsthostcount: int
    dsthostsrvcount: int
    dsthostsamesrvrate: float
    dsthostdiffsrvrate: float
    dsthostsamesrcportrate: float
    dsthostsrvdiffhostrate: float
    dsthostserrorrate: float
    dsthostsrvserrorrate: float
    dsthostrerrorrate: float
    dsthostsrvrerrorrate: float
    lastflag: int


def data_processor(df):
    df['protocoltype']=le.fit_transform(df['protocoltype'])
    df['service']=le.fit_transform(df['service'])
    df['flag']=le.fit_transform(df['flag'])
    X_test_transformed = scaler.transform(df)
    return X_test_transformed


# Define the prediction endpoint
@app.post("/predict")
async def predict(input_data: ModelInput):
    # Convert the input data to a dictionary
    input_dict = input_data.dict()
    df = pd.DataFrame([input_dict])
    X_test_transformed = data_processor(df)
    X_lstm = np.expand_dims(X_test_transformed, axis=1)  # Shape: (samples, sequence_length, features)

    # Predict using the model
    prediction = model.predict(X_lstm)
    # Define the threshold
    threshold = 0.5  # You can adjust this threshold based on your preference or requirements

    # Map the prediction to binary outcome
    if prediction > threshold:
        predicted_class = " attack "
    else:
        predicted_class = " normal traffic "

    print("Predicted class:", predicted_class)

    # Convert the prediction result to a list and return it
    return {"prediction": predicted_class}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from google.colab import userdata

auth_token = userdata.get('auth_token')
# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)

In [ ]:
{
  "duration": 0,
  "protocoltype": "tcp",
  "service": "ftp_data",
  "flag": "SF",
  "srcbytes": 491,
  "dstbytes": 0,
  "wrongfragment": 0,
  "hot": 0,
  "loggedin": 0,
  "numcompromised": 0,
  "rootshell": 0,
  "suattempted": 0,
  "numroot": 0,
  "numfilecreations": 0,
  "numshells": 0,
  "numaccessfiles": 0,
  "ishostlogin": 0,
  "isguestlogin": 0,
  "count": 0,
  "srvcount": 0,
  "serrorrate": 0,
  "srvserrorrate": 0,
  "rerrorrate": 0,
  "srvrerrorrate": 0,
  "samesrvrate": 0,
  "diffsrvrate": 0,
  "srvdiffhostrate": 0,
  "dsthostcount": 0,
  "dsthostsrvcount": 0,
  "dsthostsamesrvrate": 0.17,
  "dsthostdiffsrvrate": 0.03,
  "dsthostsamesrcportrate": 0.17,
  "dsthostsrvdiffhostrate": 0.00,
  "dsthostserrorrate": 0.00,
  "dsthostsrvserrorrate": 0.00,
  "dsthostrerrorrate": 0.05,
  "dsthostsrvrerrorrate": 0.00,
  "lastflag": 20
}










{
  "duration": 0,
  "protocoltype": "tcp",
  "service": "telnet",
  "flag": "RSTO",
  "srcbytes": 0,
  "dstbytes": 15,
  "wrongfragment": 0,
  "hot": 0,
  "loggedin": 0,
  "numcompromised": 0,
  "rootshell": 0,
  "suattempted": 0,
  "numroot": 0,
  "numfilecreations": 0,
  "numshells": 0,
  "numaccessfiles": 0,
  "ishostlogin": 0,
  "isguestlogin": 0,
  "count": 0,
  "srvcount": 0,
  "serrorrate": 0,
  "srvserrorrate": 0,
  "rerrorrate": 0,
  "srvrerrorrate": 0,
  "samesrvrate": 0,
  "diffsrvrate": 0,
  "srvdiffhostrate": 0,
  "dsthostcount": 0,
  "dsthostsrvcount": 0,
  "dsthostsamesrvrate": 0.31,
  "dsthostdiffsrvrate": 0.17,
  "dsthostsamesrcportrate": 0.03,
  "dsthostsrvdiffhostrate": 0.02,
  "dsthostserrorrate": 0.0,
  "dsthostsrvserrorrate": 0.0,
  "dsthostrerrorrate": 0.83,
  "dsthostsrvrerrorrate": 0.71,
  "lastflag": 11
}